In [1]:
# imports 

import numpy as np
import heapq
import math
import scipy as sp
from matplotlib import pyplot

In [2]:
# m         : the number of floors in the building
# n         : the number of elevators 
# lambdas   : m * m array; i,j is the rate of arrivals on floor i who want to go to floor j
# schedule  : a scheduling algorithm for the elevators
# occupancy : max occupancy of an elevator
# h         : length of simulation

# Simulation Design

# Elevators start at ground
# if button pressed, schedule should decide 
# which elevator goes and what happens in general
# ie. "soft" interrupt based system

# buttons indicate first arrival of type down/up
# therefore the amount of people that go down/up
# at elevator arrival is that minus time of first arrival

# Figure out how to incorporate non zero time to fill elevator 

# For now we will have the elevator just move up and down aimlessly

# Initialize clocks with button presses 

# down  : tuple where index j is rates of clients of floor j going down
# total : tuple where index j is sum of all rates

# Assuming that button is pressed up/down and then later the corresponding one is also pressed

# elevators: list of elevator structures

# elevator structure: clients    : list where index j is amount of people want to go to floor j
#                     next floor : next floor it goes to
#                     floor      : the current floor

# floors: list of floor structures

# floor structure: queue  : where index j is amount of people want to go to floor j,
#                  size   : amount of people on the floor
#                  button : 1/-1 to indicate up/down, 0 and size != 0 for both and size == 0 for none

# NOTE: floors are labeled 0 ... m - 1

def sim(m: int, n: int, lambdas: tuple[tuple[float]], schedule, occupancy: int, h: int):
    rng = np.random.default_rng() # rv generator
    floors = list( [np.zeros(m), 0, 0] for _ in range(m) )
    elevators = list( [np.zeros(m), 0, 0] for _ in range(n) )
    down = tuple( sum(lambdas[j][:j]) for j in range(m) )
    total = tuple( sum(lambdas[j]) for j in range(m) )
    clocks = []
    for i in range(m):
        pair = [[rng.exponential(1/total[i])]]
        # next even is a down action
        if rng.uniform(0, 1) <= down[i] / total[i]:
            pair[0].append(-1)
            pair.append(rng.exponential(1/down[i]) + pair[0])
        else:
            pair[0].append(1)
            pair.append(rng.exponential(1/(total[i] - down[i])) + pair[0])
        clocks.append(pair)
    heapq.heapify(clocks)
    t = 0           # total elapsed time

    # Elevator load from Carpark


    while(t <= h):


    return floors



IndentationError: expected an indented block after 'while' statement on line 61 (2447264516.py, line 64)